In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
Base.prepare(engine, reflect=True)

In [ ]:
# reflect the tables
Base.classes.keys()

In [ ]:
# View all of the classes that automap found
Base.classes.measurement.__table__.columns.keys()

In [ ]:
Base.classes.station.__table__.columns.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
most_recent_date

In [ ]:
# date_str = "01"
# session.query(Measurement.date).\
#     filter(func.strftime("%d", Measurement.date) == date_str).order_by(Measurement.date.desc()).limit(12).all()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
latest_date=dt.datetime.strptime(most_recent_date,"%Y-%m-%d")

# Calculate the date one year from the last date in data set.
date_one_year_back=latest_date-dt.timedelta(days=366)

# Perform a query to retrieve the data and precipitation scores
precipitation_scores=session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.asc()).\
        filter(Measurement.date>=date_one_year_back).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_scores_df = pd.DataFrame(precipitation_scores, columns=['date', 'precipitation'])
precipitation_scores_df.set_index('date', inplace=True)

# Sort the dataframe by date
precipitation_scores_df.sort_values(by=['date'])

# Use Pandas Plotting with Matplotlib to plot the data
xaxis=precipitation_scores_df.plot(color='b',figsize=(12,10),legend=True,grid=True)
plt.setp(xaxis.get_xticklabels())
plt.tight_layout()
plt.title("Precipitation Analysis")
plt.xlabel("Date")
plt.ylabel("Inches")
plt.savefig("Images/Precipitation_Analysis.png")
plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_scores_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(func.count(Station.id)).all()[0][0]
total_stations

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_count = session.query(Measurement.station, func.count(Measurement.id)).group_by(Measurement.station).\
                                     order_by(func.count(Measurement.station).desc()).all()
station_count

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = station_count[0][0]

session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), \
              func.avg(Measurement.tobs),func.count(Measurement.tobs)).\
             filter(Measurement.station==most_active_station).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperature_observation=session.query(Measurement.date, Measurement.tobs).order_by(Measurement.date.desc()).\
        filter(Measurement.station==most_active_station).\
        filter(Measurement.date>=date_one_year_back).all()

temperature_observation_df = pd.DataFrame(temperature_observation, columns=['date', 'temperature'])

# plotting the Histogram
histogram = temperature_observation_df['temperature'].hist(bins=12,figsize=(9,5), color='b', zorder=2,legend=True)
plt.tight_layout()
plt.title("Temperature Frequency")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.savefig("Images/Temperature_Frequency.png")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()